In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np
import pandas as pd
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
class FeatureRegression(nn.Module):
    def __init__(self, input_size):
        super(FeatureRegression, self).__init__()
        self.build(input_size)

    def build(self, input_size):
        self.W = Parameter(torch.Tensor(input_size, input_size))
        self.b = Parameter(torch.Tensor(input_size))

        m = torch.ones(input_size, input_size).cuda() - torch.eye(input_size, input_size).cuda()
        self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, x):
        z_h = F.linear(x, self.W * Variable(self.m), self.b)
        return z_h

class TemporalDecay(nn.Module):
    def __init__(self, input_size, output_size, diag = False):
        super(TemporalDecay, self).__init__()
        self.diag = diag

        self.build(input_size, output_size)

    def build(self, input_size, output_size):
        self.W = Parameter(torch.Tensor(output_size, input_size)).cuda()
        self.b = Parameter(torch.Tensor(output_size)).cuda()
        self.relu = nn.ReLU(inplace=False)
        if self.diag == True:
            assert(input_size == output_size)
            m = torch.eye(input_size, input_size).cuda()
            self.register_buffer('m', m)

        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.W.size(0))
        self.W.data.uniform_(-stdv, stdv)
        if self.b is not None:
            self.b.data.uniform_(-stdv, stdv)

    def forward(self, d):
        gamma = self.relu(F.linear(d, self.W, self.b))
        gamma = torch.exp(-gamma)
        return gamma

In [2]:
def make_deltas(masks):
    deltas = []
    for h in range(len(masks)):
        if h == 0:
            deltas.append([1 for _ in range(masks.shape[1])])
        else:
            deltas.append([1 for _ in range(masks.shape[1])] + (1-masks[h]) * deltas[-1])
    
    return list(deltas)


class MyDataset(Dataset):
    def __init__(self, dataset, q):
        self.data = dataset
        self.q = q

    def __len__(self):
        return self.data.shape[1] // self.q

    def __getitem__(self, index):
        return self.data[:,index * self.q : index * self.q + self.q,:]

In [4]:
df = pd.read_csv("./datasets/pm25_ground.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df_gt = pd.read_csv("./datasets/pm25_missing.csv")
df_gt['datetime'] = pd.to_datetime(df_gt['datetime'])

In [5]:
train_df = df[df['datetime'].dt.month.isin([1,2,4,5,7,8,10,11,12])]
train_df_gt = df_gt[df_gt['datetime'].dt.month.isin([1,2,4,5,7,8,10,11,12])]

In [61]:
def Air_quality(batch_size, seq_len):
    df = pd.read_csv("./datasets/pm25_ground.csv")
    df['datetime'] = pd.to_datetime(df['datetime'])
    df_gt = pd.read_csv("./datasets/pm25_missing.csv")
    df_gt['datetime'] = pd.to_datetime(df_gt['datetime'])

    rbf_df = pd.read_csv("./RBFresult/air_20_8.0_scale.csv")
    rbf_df["datetime"] = df['datetime']

    for i in [1,2,4,5,7,8,10,11,12]:
        
        train_df = df[df['datetime'].dt.month.isin([i])]
        train_df_gt = df_gt[df_gt['datetime'].dt.month.isin([i])]
        train_rbf_df = rbf_df[rbf_df['datetime'].dt.month.isin([i])]
        
        values = ((train_df[df.columns[1:]] - df[df.columns[1:]].mean()) / df[df.columns[1:]].std()).values
        shp = values.shape

        masks = ~np.isnan(values)
        masks = masks.reshape(shp)

        gt_masks = ~np.isnan(train_df_gt[df.columns[1:]].values).reshape(shp)

        deltas = np.array(make_deltas(gt_masks))
        values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
        masks = torch.from_numpy(masks).to(torch.float32)
        gt_masks = torch.from_numpy(gt_masks).to(torch.float32)
        deltas = torch.from_numpy(deltas).to(torch.float32)
        rbf_x = torch.from_numpy(train_rbf_df[df.columns[1:]].values).to(torch.float32)
        dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), gt_masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0)

        if i == 1:
            mydata = MyDataset(dataset, seq_len)
        else:
            mydata += MyDataset(dataset, seq_len)

    data = DataLoader(mydata, batch_size, shuffle=False)

    return data

In [67]:
def Air_quality_val(model):
    df = pd.read_csv("./datasets/pm25_ground.csv")
    df['datetime'] = pd.to_datetime(df['datetime'])
    df_gt = pd.read_csv("./datasets/pm25_missing.csv")
    df_gt['datetime'] = pd.to_datetime(df_gt['datetime'])

    rbf_df = pd.read_csv("./RBFresult/air_20_8.0_scale.csv")
    rbf_df["datetime"] = df['datetime']
    mae_header = 0.0
    mae_tail = 0.0
    mre_header = 0.0
    mre_tail = 0.0

    for i in [1,2,4,5,7,8,10,11,12]:
        
        train_df = df[df['datetime'].dt.month.isin([i])]
        train_df_gt = df_gt[df_gt['datetime'].dt.month.isin([i])]
        train_rbf_df = rbf_df[rbf_df['datetime'].dt.month.isin([i])]
        
        values = ((train_df[df.columns[1:]] - df[df.columns[1:]].mean()) / df[df.columns[1:]].std()).values
        shp = values.shape

        masks = ~np.isnan(values)
        masks = masks.reshape(shp)

        gt_masks = ~np.isnan(train_df_gt[df.columns[1:]].values).reshape(shp)

        deltas = np.array(make_deltas(gt_masks))
        values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
        masks = torch.from_numpy(masks).to(torch.float32)
        gt_masks = torch.from_numpy(gt_masks).to(torch.float32)
        deltas = torch.from_numpy(deltas).to(torch.float32)
        rbf_x = torch.from_numpy(train_rbf_df[df.columns[1:]].values).to(torch.float32)
        dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), gt_masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

        dataset = dataset.to(device)
        model.eval()
        loss, x_loss, back_x_loss, loss_c, bi_chat, biimputataion = model(dataset)
        
        Nonscale_imputataion = pd.DataFrame(bi_chat[0].cpu().detach() , columns= df.columns[1:])
        Nonscale_imputataion = (Nonscale_imputataion * df[df.columns[1:]].std()) + df[df.columns[1:]].mean()


        train_df = train_df.fillna(0)
        month_mae_header = np.sum(np.abs((Nonscale_imputataion.values - train_df[df.columns[1:]].values) * (masks - gt_masks).cpu().numpy()[0]))
        month_mae_tail = np.sum((masks - gt_masks).cpu().numpy())

        mae_header += month_mae_header
        mae_tail += month_mae_tail

        mre_header += np.sum(np.abs((Nonscale_imputataion.values - train_df[df.columns[1:]].values) * (masks - gt_masks).cpu().numpy()[0]))
        mre_tail +=np.sum(np.abs(train_df[df.columns[1:]].values * (masks - gt_masks).cpu().numpy()[0]))



        print(i,"month")
        print("Original MAE :", mae_header / mae_tail)
        
    print('total MAE:', mae_header / mae_tail)
    print('total MRE:', mre_header / mre_tail)

In [62]:
data = Air_quality(36, 64)

In [63]:
data

In [11]:
class MGRU4(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MGRU4, self).__init__()

        self.temp_decay_h = TemporalDecay(input_size, output_size = hidden_size, diag = False)
        self.temp_decay_x = TemporalDecay(input_size, input_size, diag = True)
        self.temp_decay_r = TemporalDecay(input_size, input_size, diag = True)
        
        self.hidden_size = hidden_size
        self.input_size = input_size

        self.build()

    def build(self):
        self.output_layer = nn.Linear(self.hidden_size, self.input_size, bias=True)
        
        self.z_layer = FeatureRegression(self.input_size)
        self.beta_layer = nn.Linear(self.input_size * 2, self.input_size)
        self.grucell = nn.GRUCell(self.input_size * 2, self.hidden_size)
        self.concat_lyaer = nn.Linear(self.input_size * 2, self.input_size)
        

    def loss(self, hat, y, m):
        return torch.sum(torch.abs((y - hat)) * m) / (torch.sum(m) + 1e-5)

    
    def forward(self, input):
        values = input[:,0,::]
        delta = input[:,1,::]
        masks = input[:,2,::]
        gt_masks = input[:,3,::]
        rbfs = input[:,4,::]

        hid = torch.zeros((values.size(0), self.hidden_size)).cuda()

        x_loss = 0.0
        x_hat_loss = 0.0
        concat_loss = 0.0
        z_hat_loss = 0.0 
        c_hat_loss = 0.0
        imputations = []
        c_hat_list = []
        for i in range(values.size(1)):

            v = values[:,i,:]
            d = delta[:,i,:]
            m = masks[:,i,:]
            gt_m = gt_masks[:,i,:]
            r = rbfs[:,i,:]

            gamma_x = self.temp_decay_x(d)
            gamma_h = self.temp_decay_h(d)

            
            x_hat = self.output_layer(hid)
            x_loss += torch.sum(torch.abs(v - x_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)
            x_hat_loss += torch.sum(torch.abs(v - x_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)
            x_c = gt_m * v + (1 - gt_m) * x_hat

            RG = torch.cat([x_c, r], dim = 1)
            concat_hat = self.concat_lyaer(RG)
            x_loss += torch.sum(torch.abs(v - concat_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)
            concat_loss += torch.sum(torch.abs(v - concat_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)
            a_c = gt_m * v + (1 - gt_m) * concat_hat

            z_hat = self.z_layer(a_c)
            x_loss += torch.sum(torch.abs(v - z_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)
            z_hat_loss += torch.sum(torch.abs(v - z_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)

            beta_weight = torch.cat([gamma_x, gt_m], dim = 1)
            beta = torch.sigmoid(self.beta_layer(beta_weight))

            c_hat = beta * z_hat + (1 - beta) * x_hat
            x_loss += torch.sum(torch.abs(v - c_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)
            c_hat_loss += torch.sum(torch.abs(v - c_hat) * (m - gt_m)) / (torch.sum((m - gt_m)) + 1e-5)

            c_c = gt_m * v + (1 - gt_m) * c_hat

            gru_input = torch.cat([c_c, gt_m], dim = 1)
            imputations.append(c_c.unsqueeze(dim = 1))
            c_hat_list.append(c_hat.unsqueeze(1))
            
            # GRU cell
            hid = hid * gamma_h
            hid = self.grucell(gru_input, hid)

        c_hat_list = torch.cat(c_hat_list, dim = 1)
        imputations = torch.cat(imputations, dim = 1)
        return c_hat_list, imputations, x_loss * 5, x_hat_loss , concat_loss , z_hat_loss , c_hat_loss 

In [12]:
class BiMGRU4(nn.Module):
    def __init__(self, input_size, hidden_size, bias = True):
        super(BiMGRU4, self).__init__()
        
        self.fmGRU = MGRU4(input_size, hidden_size)
        self.bmGRU = MGRU4(input_size, hidden_size)
    
    def get_consistency_loss(self, pred_f, pred_b):
        loss = torch.pow(pred_f - pred_b, 2.0).mean()
        return loss
    
    def backdirect_data(self, tensor_):
        if tensor_.dim() <= 1:
            return tensor_
    
        indices = range(tensor_.size()[2])[::-1]
        indices = Variable(torch.LongTensor(indices), requires_grad = False)

        if torch.cuda.is_available():
            indices = indices.cuda()

        return tensor_.index_select(2, indices)
    
    def backdirect_imputation(self, tensor_):
        if tensor_.dim() <= 1:
            return tensor_
    
        indices = range(tensor_.size()[1])[::-1]
        indices = Variable(torch.LongTensor(indices), requires_grad = False)

        if torch.cuda.is_available():
            indices = indices.cuda()

        return tensor_.index_select(1, indices)

    def forward(self, dataset):
        back_dataset = self.backdirect_data(dataset)

        c_hat_list, x_imputataions, x_loss, x_hat_loss, concat_loss, z_hat_loss, c_hat_loss = self.fmGRU(dataset)
        back_c_hat_list, back_x_imputataions, back_x_loss, back_x_hat_loss, back_concat_loss, back_z_hat_loss, back_c_hat_loss = self.bmGRU(back_dataset)

        loss_c = self.get_consistency_loss(c_hat_list, self.backdirect_imputation(back_c_hat_list))
        loss = x_loss + back_x_loss + loss_c

        bi_c_hat = (c_hat_list +  self.backdirect_imputation(back_c_hat_list)) / 2
        bi_imputation = (x_imputataions +  self.backdirect_imputation(back_x_imputataions)) / 2

        return  loss, x_loss, back_x_loss, loss_c, bi_c_hat, bi_imputation


In [13]:
def train_BiMGRU(model, lr, epochs, dataset, device):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    progress = tqdm(range(epochs))
    
    imputation_list = []
    loss_list = []
    model.to(device)

    for epoch in progress:
        batch_loss = 0.0
        batch_f_loss = 0.0
        batch_b_loss = 0.0
        batch_c_loss = 0.0
        for data in dataset:
            data = data.to(device)
            loss, x_loss, back_x_loss, loss_c, bi_chat, biimputataion = model(data)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            imputation_list.append(bi_chat)

            batch_loss += loss
            batch_f_loss += x_loss
            batch_c_loss += loss_c
            batch_b_loss += back_x_loss
            
        progress.set_description("loss: {}, f_MGRU loss : {}, b_MGRU loss : {}, consistency Loss : {}".format(batch_loss, batch_f_loss, batch_b_loss, batch_c_loss))
        loss_list.append(batch_loss)

    return loss_list

In [66]:
G = BiMGRU4(36, 64)
loss_list = train_BiMGRU(G, 0.001, 10, data, device)

  0%|          | 0/10 [00:00<?, ?it/s]

loss: 2947.701904296875, f_MGRU loss : 1475.9337158203125, b_MGRU loss : 1471.0772705078125, consistency Loss : 0.6909888982772827: 100%|██████████| 10/10 [00:20<00:00,  2.05s/it]


In [68]:
Air_quality_val(G)

1 month
Original MAE : 31.72790611957437
2 month
Original MAE : 30.602929229074686
4 month
Original MAE : 25.57732636277664
5 month
Original MAE : 22.413253422055018
7 month
Original MAE : 22.40539075832506
8 month
Original MAE : 21.672982378945413
10 month
Original MAE : 22.25887801349199
11 month
Original MAE : 24.562156857003636
12 month
Original MAE : 25.529381449254693
total MAE: 25.529381449254693
total MRE: 0.35477356400009785


In [69]:
def Air_quality_test(model):
    df2 = pd.read_csv("./datasets/pm25_ground.csv")
    df = pd.read_csv("./datasets/pm25_missing.csv")
    df['datetime'] = pd.to_datetime(df['datetime'])
    df_gt = pd.read_csv("./datasets/pm25_missing2.csv")
    df_gt['datetime'] = pd.to_datetime(df_gt['datetime'])

    rbf_df = pd.read_csv("./RBFresult/air_20_8.0_scale.csv")
    rbf_df["datetime"] = df['datetime']
    mae_header = 0.0
    mae_tail = 0.0
    mre_header = 0.0
    mre_tail = 0.0

    for i in [3,6,9,12]:
        
        train_df = df[df['datetime'].dt.month.isin([i])]
        train_df_gt = df_gt[df_gt['datetime'].dt.month.isin([i])]
        train_rbf_df = rbf_df[rbf_df['datetime'].dt.month.isin([i])]
        
        values = ((train_df[df.columns[1:]] - df2[df.columns[1:]].mean()) / df2[df.columns[1:]].std()).values
        shp = values.shape

        masks = ~np.isnan(values)
        masks = masks.reshape(shp)

        gt_masks = ~np.isnan(train_df_gt[df.columns[1:]].values).reshape(shp)

        deltas = np.array(make_deltas(gt_masks))
        values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
        masks = torch.from_numpy(masks).to(torch.float32)
        gt_masks = torch.from_numpy(gt_masks).to(torch.float32)
        deltas = torch.from_numpy(deltas).to(torch.float32)
        rbf_x = torch.from_numpy(train_rbf_df[df.columns[1:]].values).to(torch.float32)
        dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), gt_masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

        dataset = dataset.to(device)
        model.eval()
        loss, x_loss, back_x_loss, loss_c, bi_chat, biimputataion = model(dataset)
        
        Nonscale_imputataion = pd.DataFrame(bi_chat[0].cpu().detach() , columns= df.columns[1:])
        Nonscale_imputataion = (Nonscale_imputataion * df2[df.columns[1:]].std()) + df2[df.columns[1:]].mean()


        train_df = train_df.fillna(0)
        month_mae_header = np.sum(np.abs((Nonscale_imputataion.values - train_df[df.columns[1:]].values) * (masks - gt_masks).cpu().numpy()[0]))
        month_mae_tail = np.sum((masks - gt_masks).cpu().numpy())

        mae_header += month_mae_header
        mae_tail += month_mae_tail

        mre_header += np.sum(np.abs((Nonscale_imputataion.values - train_df[df.columns[1:]].values) * (masks - gt_masks).cpu().numpy()[0]))
        mre_tail +=np.sum(np.abs(train_df[df.columns[1:]].values * (masks - gt_masks).cpu().numpy()[0]))

        print(i,"month")
        print("Original MAE :", mae_header / mae_tail)
        
    print('total MAE:', mae_header / mae_tail)
    print('total MRE:', mre_header / mre_tail)

In [70]:
Air_quality_test(G)

3 month
Original MAE : 30.34336988346258
6 month
Original MAE : 24.729781385048703
9 month
Original MAE : 24.106634840709543
12 month
Original MAE : 25.631015952001782
total MAE: 25.631015952001782
total MRE: 0.3872111825348534


In [55]:
Air_quality_test(G)

Original MAE : 23.792308877487844
Original MRE : 0.3594335890913857


In [54]:
def Air_quality_test(model):
    df = pd.read_csv("./datasets/pm25_missing.csv")
    df['datetime'] = pd.to_datetime(df['datetime'])
    df_gt = pd.read_csv("./datasets/pm25_missing2.csv")
    df_gt['datetime'] = pd.to_datetime(df_gt['datetime'])

    rbf_df = pd.read_csv("./RBFresult/air_20_8.0_scale.csv")
    rbf_df["datetime"] = df['datetime']

    train_df = df
    train_df_gt = df_gt
    train_rbf_df = rbf_df
    
    values = ((train_df[df.columns[1:]] - df[df.columns[1:]].mean()) / df[df.columns[1:]].std()).values
    shp = values.shape

    masks = ~np.isnan(values)
    masks = masks.reshape(shp)

    gt_masks = ~np.isnan(train_df_gt[df.columns[1:]].values).reshape(shp)

    deltas = np.array(make_deltas(gt_masks))
    values = torch.nan_to_num(torch.from_numpy(values).to(torch.float32))
    masks = torch.from_numpy(masks).to(torch.float32)
    gt_masks = torch.from_numpy(gt_masks).to(torch.float32)
    deltas = torch.from_numpy(deltas).to(torch.float32)
    rbf_x = torch.from_numpy(train_rbf_df[df.columns[1:]].values).to(torch.float32)
    dataset = torch.cat([values.unsqueeze_(0), deltas.unsqueeze_(0), masks.unsqueeze_(0), gt_masks.unsqueeze_(0), rbf_x.unsqueeze_(0)], dim = 0).unsqueeze_(0)

    dataset = dataset.to(device)
    model.eval()
    loss, x_loss, back_x_loss, loss_c, bi_chat, biimputataion = model(dataset)

    Nonscale_imputataion = pd.DataFrame(bi_chat[0].cpu().detach() , columns= df.columns[1:])
    Nonscale_imputataion = (Nonscale_imputataion * df[df.columns[1:]].std()) + df[df.columns[1:]].mean()


    train_df = train_df.fillna(0)

    print("Original MAE :", np.sum(np.abs((Nonscale_imputataion.values - train_df[df.columns[1:]].values) * (masks - gt_masks).cpu().numpy()[0])) / np.sum((masks - gt_masks).cpu().numpy()))
    print("Original MRE :", np.sum(np.abs((Nonscale_imputataion.values - train_df[df.columns[1:]].values) * (masks - gt_masks).cpu().numpy()[0])) / np.sum(np.abs(train_df[df.columns[1:]].values * (masks - gt_masks).cpu().numpy()[0])))